In [ ]:
import src.cctbx_dep.phase_from_cif as pfc

## Load up a phase structure
And show scatterers

In [ ]:
phase_settings = {
   'cif_path': '.\ecm-occ\Cif Cards\Al-Si order-disorder\Microcline.cif'
}
phase = pfc.load_cif(phase_settings)
struct = phase['structure']
uc = phase['unit_cell']
print phase['unit_cell'].parameters()
print struct.show_scatterers()
#    scat.flags.set_grad_site(True)
#    print scat.flags.grad_site()
#    print dir(scat.flags)

In [ ]:
print struct.non_unit_occupancy_implies_min_distance_sym_equiv_zero()

## Occupancy sum isn't preserved in cctbx.xray...

In [ ]:
mask = struct.by_index_selection([5,6])
print mask.as_numpy_array()
struct2 = struct.select(mask)
struct2.show_scatterers()
mask_Si = struct2.by_index_selection([0])
struct2.set_occupancies(1.0, mask_Si)
struct2.show_scatterers()

## Does mmtbx do better?

In [ ]:
pdb_string = struct.as_pdb_file()
from iotbx.pdb import hierarchy
pdb_in = hierarchy.input(pdb_string=pdb_string)
import mmtbx.refinement.occupancies as mmtbx_occ
print mmtbx_occ.extract_partial_occupancy_selections(pdb_in.hierarchy)
import mmtbx
# help(mmtbx.utils.experimental_data_target_and_gradients)
# mmtbx.utils.experimental_data_target_and_gradients(struct.structure_factors(d_min=1).f_calc())

In [ ]:
xray_structure = pdb_in.xray_structure_simple()
# make up "F-obs" and R-free-flags
f_obs = abs(xray_structure.structure_factors(d_min=1.5).f_calc())
r_free_flags = f_obs.generate_r_free_flags()
# instantiate fmodel object
fmodel = mmtbx.f_model.manager(
 f_obs          = f_obs,
 r_free_flags   = r_free_flags,
 xray_structure = xray_structure)
# now we can caclualte R-factors and show some statistics
print "r_work:", fmodel.r_work()
print "r_free:", fmodel.r_free()
fmodel.show()

In [ ]:
xrs = pdb_in.xray_structure_simple()
xrs.scattering_type_registry(table = "wk1995")
#
# good = """
# CRYST1   15.000   15.000   15.000  90.00  90.00  90.00 P 212121
# HETATM  115  O   HOH A  18       3.000   5.000   5.000  1.00 10.00           O
# HETATM  115  O   HOH A  18       5.000   5.000   8.000  1.00 10.00           O
# TER
# END
# """
# xb = iotbx.pdb.input(source_info=None, lines=good).xray_structure_simple()
# xb.scattering_type_registry(table = "wk1995")
# f_obs = abs(xb.structure_factors(d_min=1,
# algorithm="direct").f_calc()).set_observation_type_xray_amplitude()
# #
sfp = mmtbx.f_model.sf_and_grads_accuracy_master_params.extract()
sfp.algorithm="direct"
for target_name in ["ls_wunit_kunit", "ls_wunit_k1", "ml"]:
   print "%s"%target_name, "-"*50
   fmodel = mmtbx.f_model.manager(
     f_obs                        = f_obs,
     xray_structure               = xrs.deep_copy_scatterers(),
     target_name                  = target_name,
     sf_and_grads_accuracy_params = sfp)
   fmodel.update_all_scales()
   alpha_beta = fmodel.alpha_beta()
   print "R-work: %6.4f"%fmodel.r_work()
   tg = mmtbx.utils.experimental_data_target_and_gradients(fmodel = fmodel,
     alpha_beta=alpha_beta)
   tg.show()
   eps = 1.e-6
   # occupancies
   go = tg.grad_occ()
   for i in [0,1]:
     xrs1 = fmodel.xray_structure.deep_copy_scatterers()
   #   xrs2 = fmodel.xray_structure.deep_copy_scatterers()
   #   #
     xrs1.scatterers()[i].occupancy+=+eps
     tg.update_xray_structure(xray_structure=xrs1, alpha_beta=alpha_beta)
     t1 = tg.target()
     print t1

In [ ]:
print dir(fmodel)

In [ ]:
mmtbx.fmodels(fmodel_xray=fmodel)

In [ ]:
import cctbx.array_family.flex as flex

In [ ]:
print flex.bool([True, True]).size()

In [ ]:
print help(struct.select([True]*len(struct)))
# print struct[5:]
# for ind in [5, 6]:
#    print struct[ind]
#    print dir(scat)
#    print scat.show(unit_cell=uc)

## smtbx attempt

In [ ]:
from __future__ import division, absolute_import
from libtbx import object_oriented_patterns as oop
from scitbx.linalg import eigensystem, svd
from scitbx import matrix
from scitbx.lstbx import normal_eqns_solving
from cctbx import sgtbx, crystal, xray, adptbx, uctbx
import cctbx.sgtbx.lattice_symmetry
from cctbx import euclidean_model_matching as emma
from cctbx.array_family import flex
from cctbx.development import random_structure
from smtbx.refinement import least_squares
from smtbx.refinement import constraints
from smtbx.refinement.restraints import origin_fixing_restraints
import smtbx.utils
from libtbx.test_utils import approx_equal
import libtbx.utils
import math
import sys
import random
import numpy as np

class special_positions_test(object):

  delta_site   = 0.1 # % (of unit cell c for constrained atoms)
  delta_u_star = 0.1 # %

  def __init__(self, n_runs, **kwds):
    libtbx.adopt_optional_init_args(self, kwds)
    self.n_runs = n_runs
    self.crystal_symmetry = crystal.symmetry(
      unit_cell=uctbx.unit_cell((5.1534, 5.1534, 8.6522, 90, 90, 120)),
      space_group_symbol='Hall: P 6c')
    self.structure = xray.structure(
      self.crystal_symmetry.special_position_settings(),
      flex.xray_scatterer((
        xray.scatterer('K1',
                        site=(0, 0, -0.00195),
                        u=self.u_cif_as_u_star((0.02427, 0.02427, 0.02379,
                                                0.01214, 0.00000, 0.00000))),
        xray.scatterer('S1',
                       site=(1/3, 2/3, 0.204215),
                       u=self.u_cif_as_u_star((0.01423, 0.01423, 0.01496,
                                               0.00712, 0.00000, 0.00000 ))),
        xray.scatterer('Li1',
                       site=(1/3, 2/3, 0.815681),
                       u=self.u_cif_as_u_star((0.02132, 0.02132, 0.02256,
                                               0.01066, 0.00000, 0.00000 ))),
        xray.scatterer('O1',
                       site=(1/3, 2/3, 0.035931),
                       u=self.u_cif_as_u_star((0.06532, 0.06532, 0.01669,
                                               0.03266, 0.00000, 0.00000 ))),
        xray.scatterer('O2',
                       site=(0.343810, 0.941658, 0.258405),
                       u=self.u_cif_as_u_star((0.02639,  0.02079, 0.05284,
                                               0.01194, -0.00053,-0.01180 )))
      )))
    mi = self.crystal_symmetry.build_miller_set(anomalous_flag=False,
                                                d_min=0.5)
    fo_sq = mi.structure_factors_from_scatterers(
      self.structure, algorithm="direct").f_calc().norm()
    self.fo_sq = fo_sq.customized_copy(sigmas=flex.double(fo_sq.size(), 1))

  def u_cif_as_u_star(self, u_cif):
    return adptbx.u_cif_as_u_star(self.crystal_symmetry.unit_cell(), u_cif)

  def shake_point_group_3(self, sc):
    _, _, c, _, _, _ = self.crystal_symmetry.unit_cell().parameters()

    x, y, z = sc.site
    z += random.uniform(-self.delta_site, self.delta_site)/c
    sc.site = (x, y, z)

    u11, _, u33, _, _, _ = sc.u_star
    u11 *= 1 + random.uniform(-self.delta_u_star, self.delta_u_star)
    u33 *= 1 + random.uniform(-self.delta_u_star, self.delta_u_star)
    sc.u_star = (u11, u11, u33, u11/2, 0, 0)

  def run(self):
    if self.n_runs > 1:
      print 'small inorganic refinement with many special positions'
      for i in xrange(self.n_runs):
        print '.',
        self.exercise()
      print
    else:
      self.exercise()

  def exercise(self):
    xs0 = self.structure
    xs = xs0.deep_copy_scatterers()
    k1, s1, li1, o1, o2 = xs.scatterers()
    self.shake_point_group_3(k1)
    self.shake_point_group_3(s1)
    self.shake_point_group_3(li1)
    self.shake_point_group_3(o1)
    o2.site = tuple(
      [ x*(1 + random.uniform(-self.delta_site, self.delta_site))
        for x in o2.site])
    o2.u_star = tuple(
      [ u*(1 + random.uniform(-self.delta_u_star, self.delta_u_star))
        for u in o2.u_star])

    for sc in xs.scatterers():
      sc.flags.set_use_u_iso(False).set_use_u_aniso(True)
      sc.flags.set_grad_site(True).set_grad_u_aniso(True)
      connectivity_table = smtbx.utils.connectivity_table(xs)
    reparametrisation = constraints.reparametrisation(
      structure=xs,
      constraints=[],
      connectivity_table=connectivity_table)
    x = self.fo_sq.as_xray_observations().indices.as_vec3_double().as_double().as_numpy_array().reshape(-1, 3)
#     print dir(self.fo_sq.as_xray_observations().fo_sq.as_double())
    y = self.fo_sq.as_xray_observations().fo_sq.as_double().data().as_numpy_array()
    print x.shape, y.shape
    for item in zip(list(x), list(y)):
      print item
    ls = least_squares.crystallographic_ls(
      self.fo_sq.as_xray_observations(), reparametrisation,
      weighting_scheme=least_squares.unit_weighting(),
      origin_fixing_restraints_type=
      origin_fixing_restraints.atomic_number_weighting)

    cycles = normal_eqns_solving.levenberg_marquardt_iterations(
      ls,
      gradient_threshold=1e-12,
      step_threshold=1e-7,
      track_all=True)

    ## Test whether refinement brought back the shaked structure to its
    ## original state
    match = emma.model_matches(xs0.as_emma_model(),
                               xs.as_emma_model()).refined_matches[0]
    assert match.rt.r == matrix.identity(3)
    assert not match.singles1 and not match.singles2
    assert match.rms < 1e-6

    delta_u_carts= (   xs.scatterers().extract_u_cart(xs.unit_cell())
                    - xs0.scatterers().extract_u_cart(xs.unit_cell())).norms()
    assert flex.abs(delta_u_carts) < 1e-6

    assert approx_equal(ls.scale_factor(), 1, eps=1e-4)

    ## Test covariance matrix
    jac_tr = reparametrisation.jacobian_transpose_matching_grad_fc()
    cov = ls.covariance_matrix(
      jacobian_transpose=jac_tr, normalised_by_goof=False)\
        .matrix_packed_u_as_symmetric()
    m, n = cov.accessor().focus()
    # x,y for point group 3 sites are fixed: no variance or correlation
    for i in (0, 9, 18, 27,):
      assert cov.matrix_copy_block(i, 0, 2, n) == 0

    # u_star coefficients u13 and u23 for point group 3 sites are fixed
    # to 0: again no variance or correlation with any other param
    for i in (7, 16, 25, 34,):
      assert cov.matrix_copy_block(i, 0, 2, n).as_1d()\
             .all_approx_equal(0., 1e-20)

    # u_star coefficients u11, u22 and u12 for point group 3 sites
    # are totally correlated, with variances in ratios 1:1:1/2
    for i in (3, 12, 21, 30,):
      assert cov[i, i] != 0
      assert approx_equal(cov[i, i], cov[i+1, i+1], eps=1e-15)
      assert approx_equal(cov[i, i+1]/cov[i, i], 1, eps=1e-12)
      assert approx_equal(cov[i, i+3]/cov[i, i], 0.5, eps=1e-12)

In [ ]:
special_positions_test(1).exercise()